In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical



In [4]:
df = pd.read_csv('e:\Work\huda\output.csv', low_memory=False)

df.head(1)

,id,student_name,student_email,student_gender,student_age,student_course,student_degree,course_id,enrolled_course
0,1,Gloria Stone,gloria.stone@outlook.com,Male,19,Computer Science & Information Technology,Doctorate Degree,"['MN-2661', 'DS-4771']",DS-4771


In [5]:
df = df.drop(['student_gender', 'student_age', 'student_name', 'student_email'], axis=1)
df.head(1)

,id,student_course,student_degree,course_id,enrolled_course
0,1,Computer Science & Information Technology,Doctorate Degree,"['MN-2661', 'DS-4771']",DS-4771


In [6]:
import ast

df.course_id = df.course_id.apply(ast.literal_eval)

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

s = df['course_id']

mlb = MultiLabelBinarizer()

course_dummies = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

In [8]:
df_basic = df[['id', 'course_id', 'enrolled_course', 'student_course', 'student_degree']]
df_new = pd.concat([df_basic, course_dummies], axis=1)
df_new = pd.get_dummies(df_new, columns=['student_course', 'student_degree'])
df_new.head(1)

,id,course_id,enrolled_course,BA-5120,BA-6633,BA-7701,BA-8090,BA-8551,BA-9801,BB-1629,...,student_course_Computer Science and Arts,student_course_Education,student_course_Engineering,student_course_Law,student_course_Medicine,student_course_Nursing,student_course_Science,student_degree_Bachelor Degree,student_degree_Doctorate Degree,student_degree_Master Degree
0,1,"[MN-2661, DS-4771]",DS-4771,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
from sklearn.preprocessing import LabelEncoder

X = df_new.drop(['course_id', 'id', 'enrolled_course'], axis=1)

encoder = LabelEncoder()
Y = pd.get_dummies(encoder.fit_transform(df_new.enrolled_course)).values
y = df_new.enrolled_course

In [10]:
encoder.classes_

array(['BA-5120', 'BA-6633', 'BA-7701', 'BA-8090', 'BA-8551', 'BA-9801',
       'BB-1629', 'BB-5511', 'BC-0025', 'BC-4400', 'BC-5759', 'BC-7484',
       'BD-1110', 'BD-1199', 'BD-4000', 'BD-4878', 'BD-7375', 'BE-2983',
       'BE-3311', 'BE-5421', 'BF-3399', 'BF-7263', 'BH-7020', 'BL-2244',
       'BL-7247', 'BM-0033', 'BM-0203', 'BM-2211', 'BM-2324', 'BM-7543',
       'BM-7565', 'BN-2277', 'BN-5950', 'BN-7775', 'BQ-5533', 'BR-1790',
       'BR-5867', 'BS-0318', 'BS-1818', 'BS-6070', 'BS-7230', 'BS-8990',
       'BT-1144', 'BV-0035', 'BV-1199', 'BV-4565', 'BZ-6600', 'CA-2090',
       'CD-8321', 'CP-2047', 'CS-2314', 'CS-8976', 'DA-4499', 'DA-7979',
       'DC-4639', 'DC-5050', 'DC-8010', 'DD-7565', 'DE-0102', 'DE-2211',
       'DE-2910', 'DL-1001', 'DL-1090', 'DM-0099', 'DM-7571', 'DQ-3535',
       'DS-4771', 'DS-5456', 'DS-7771', 'DT-6020', 'DW-2456', 'DX-3738',
       'EN-6721', 'MA-1254', 'MA-2266', 'MA-3601', 'MA-6050', 'MA-6112',
       'MA-7693', 'MA-8221', 'MB-0501', 'MB-7766', 

In [11]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=21)
_, _, _, _y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [13]:
len(X_train.columns)

144

In [14]:
# pretrained_model = tf.keras.applications.ResNet50(include_top=False)
from tensorflow.keras import regularizers

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(144,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),
    # pretrained_model,
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(131, activation='softmax')
  ])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, batch_size=50, epochs=15)

Epoch 1/15
150/150 [==============================] - 2s 2ms/step - loss: 4.9737 - accuracy: 0.0147
Epoch 2/15
150/150 [==============================] - 0s 2ms/step - loss: 4.8347 - accuracy: 0.0351
Epoch 3/15
150/150 [==============================] - 0s 1ms/step - loss: 4.5952 - accuracy: 0.0760
Epoch 4/15
150/150 [==============================] - 0s 1ms/step - loss: 4.3280 - accuracy: 0.1151
Epoch 5/15
150/150 [==============================] - 0s 1ms/step - loss: 4.1299 - accuracy: 0.1387
Epoch 6/15
150/150 [==============================] - 0s 1ms/step - loss: 3.9508 - accuracy: 0.1789
Epoch 7/15
150/150 [==============================] - 0s 2ms/step - loss: 3.8539 - accuracy: 0.1856
Epoch 8/15
150/150 [==============================] - 0s 1ms/step - loss: 3.7661 - accuracy: 0.2068
Epoch 9/15
150/150 [==============================] - 0s 2ms/step - loss: 3.7035 - accuracy: 0.2148
Epoch 10/15
150/150 [==============================] - 0s 1ms/step - loss: 3.6385 - accuracy: 0.2247

In [16]:
y_pred = model.predict(X_test)
y_pred_class = np.argmax(model.predict(X_test), axis=-1)
y_pred_class

79/79 [==============================] - 0s 1ms/step


array([ 78,  13, 125, ...,  60, 111,  28], dtype=int64)

In [17]:
print(y_pred_class)

[ 78  13 125 ...  60 111  28]


In [18]:
y_pred_class_labels = encoder.inverse_transform(y_pred_class)
y_pred_class_labels

array(['MA-7693', 'BD-1199', 'MT-6325', ..., 'DE-2910', 'MO-6050',
       'BM-2324'], dtype=object)

In [19]:
from sklearn.metrics import classification_report
report = pd.DataFrame(classification_report(_y_test, y_pred_class_labels, output_dict=True)).transpose()

e:\venvs\ds1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\venvs\ds1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\venvs\ds1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
report.to_csv('dl_cm_student_course_degree.csv')

In [20]:
report

,precision,recall,f1-score,support
BA-5120,0.555556,1.000000,0.714286,20.0000
BA-6633,0.625000,0.500000,0.555556,20.0000
BA-7701,0.000000,0.000000,0.000000,24.0000
BA-8090,0.650000,0.619048,0.634146,21.0000
BA-8551,0.406250,1.000000,0.577778,13.0000
...,...,...,...,...
MZ-6611,0.380952,0.800000,0.516129,10.0000
MZ-9600,0.444444,0.400000,0.421053,20.0000
accuracy,0.474400,0.474400,0.474400,0.4744
macro avg,0.467901,0.480935,0.420737,2500.0000
